In [1]:
import pandas as pd

# Load the dataset
file_name = 'met_museum_objects.csv'  # Replace 'my_data.csv' with your actual filename
df = pd.read_csv(file_name)

# 1. Display the first 5 rows
print("--- Head of the DataFrame ---")
print(df.head())

# 2. Display the summary information (column types, non-null counts)
print("\n--- DataFrame Information ---")
print(df.info())

/var/folders/rk/20yydmvn1bnd2drlzkvx8y6c0000gn/T/ipykernel_14707/1669775151.py:5: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


--- Head of the DataFrame ---
  Object Number  Is Highlight  Is Public Domain  Object ID  \
0    1979.486.1         False             False          1   
1    1980.264.5         False             False          2   
2      67.265.9         False             False          3   
3     67.265.10         False             False          4   
4     67.265.11         False             False          5   

                 Department Object Name                         Title Culture  \
0  American Decorative Arts        Coin  One-dollar Liberty Head Coin     NaN   
1  American Decorative Arts        Coin  Ten-dollar Liberty Head Coin     NaN   
2  American Decorative Arts        Coin    Two-and-a-Half Dollar Coin     NaN   
3  American Decorative Arts        Coin    Two-and-a-Half Dollar Coin     NaN   
4  American Decorative Arts        Coin    Two-and-a-Half Dollar Coin     NaN   

  Period Dynasty  ... Subregion Locale Locus Excavation River Classification  \
0    NaN     NaN  ...       Na

In [2]:
import pandas as pd

# --- Missing Values Analysis (Data Completeness) ---
print("\n" + "="*50)
print("STEP 2: Data Completeness - Missing Values Analysis")
print("="*50)

# Calculate the percentage of missing values
missing_data = df.isnull().sum()
missing_percentage = (missing_data[missing_data > 0] / len(df)) * 100

# Sort and display the columns with missing data, descending
missing_report = pd.DataFrame({
    'Missing Count': missing_data[missing_data > 0],
    'Missing Percentage': missing_percentage.round(2)
}).sort_values(by='Missing Percentage', ascending=False)

print(missing_report)


# --- Key Categorical Column Distribution (Data Profiling) ---
print("\n" + "="*50)
print("STEP 3: Data Profiling - Department Distribution")
print("="*50)

# Analyze the 'Department' column, which is fully populated and essential for segmentation.
department_counts = df['Department'].value_counts()
print(f"Total Unique Departments: {len(department_counts)}\n")
print("Top 10 Departments by Object Count:")
print(department_counts.head(10))


STEP 2: Data Completeness - Missing Values Analysis
                         Missing Count  Missing Percentage
River                           446100               99.53
County                          445715               99.44
Locus                           441264               98.45
State                           439843               98.13
Artist Suffix                   437991               97.72
Reign                           437386               97.59
Locale                          433108               96.63
Excavation                      432684               96.54
Portfolio                       427833               95.46
Subregion                       426487               95.15
Rights and Reproduction         425228               94.87
Dynasty                         425185               94.86
City                            417683               93.19
Region                          417125               93.07
Geography Type                  389740               86.96
Per

In [3]:
# --- Outlier Detection (Numerical Date Fields) ---
print("\n" + "="*50)
print("STEP 4: Object Date Analysis (Outlier Check)")
print("="*50)

# 1. Descriptive Statistics on numerical date fields
date_cols = ['Object Begin Date', 'Object End Date']
print("Descriptive Statistics for Object Dates:")
print(df[date_cols].describe())

# 2. Check for suspicious minimum/maximum dates (potential Outliers)
print("\nObjects with the 5 Earliest Begin Dates:")
# Filter for the earliest 5 unique dates
earliest_dates = df.sort_values('Object Begin Date').head(5)
print(earliest_dates[['Object Begin Date', 'Title', 'Department']])

print("\nObjects with the 5 Latest End Dates:")
# Filter for the latest 5 unique dates
latest_dates = df.sort_values('Object End Date', ascending=False).head(5)
print(latest_dates[['Object End Date', 'Title', 'Department']])


STEP 4: Object Date Analysis (Outlier Check)
Descriptive Statistics for Object Dates:
       Object Begin Date  Object End Date
count       4.482030e+05     4.482030e+05
mean        1.390817e+03     1.493446e+03
std         2.779816e+04     2.778712e+04
min        -9.500000e+04    -3.500000e+04
25%         1.567000e+03     1.600000e+03
50%         1.800000e+03     1.850000e+03
75%         1.895000e+03     1.910000e+03
max         1.859186e+07     1.859186e+07

Objects with the 5 Earliest Begin Dates:
        Object Begin Date       Title    Department
379043             -95000  Flake Tool  Egyptian Art
379047             -90000   Arrowhead  Egyptian Art
379039             -90000   Arrowhead  Egyptian Art
379050             -90000   Arrowhead  Egyptian Art
379042             -90000   Arrowhead  Egyptian Art

Objects with the 5 Latest End Dates:
        Object End Date                                              Title  \
400509         18591861  Left hand side of sheet 'The world upsid

In [4]:
# --- Check Integrity within the Artist Block ---
print("\n" + "="*50)
print("STEP 5: Checking Data Dependencies (Artist Block Integrity)")
print("="*50)

# Define the columns to check dependence upon the Artist Display Name
driver_col = 'Artist Display Name'
dependent_cols = [
    'Artist Alpha Sort',
    'Artist Nationality',
    'Artist Begin Date',
    'Artist End Date',
    'Artist Display Bio'
]

# 1. Filter the DataFrame where the driver column is NOT null
df_artist_exists = df[df[driver_col].notnull()]
total_artist_exists = len(df_artist_exists)

if total_artist_exists > 0:
    print(f"Total rows where '{driver_col}' exists: {total_artist_exists}\n")

    # 2. Calculate the completeness of the dependent columns in this subset
    dependency_check = {}
    for col in dependent_cols:
        count = df_artist_exists[col].notnull().sum()
        percentage = (count / total_artist_exists) * 100
        dependency_check[col] = f"{count} ({percentage:.2f}%)"

    # 3. Print the results
    print("Completeness of dependent columns WHEN Artist Display Name EXISTS:")
    for col, value in dependency_check.items():
        print(f"- {col}: {value} complete")

else:
    print(f"The driving column '{driver_col}' has no non-null entries to check dependencies.")


STEP 5: Checking Data Dependencies (Artist Block Integrity)
Total rows where 'Artist Display Name' exists: 261111

Completeness of dependent columns WHEN Artist Display Name EXISTS:
- Artist Alpha Sort: 261088 (99.99%) complete
- Artist Nationality: 196132 (75.11%) complete
- Artist Begin Date: 215234 (82.43%) complete
- Artist End Date: 212825 (81.51%) complete
- Artist Display Bio: 224064 (85.81%) complete


In [5]:
from sklearn.neighbors import LocalOutlierFactor
import numpy as np

print("\n" + "="*50)
print("STEP 6: Outlier Identification and Inspection (LOF Concept)")
print("="*50)

# 1. Focus on the suspicious Object End Date
suspicious_max_date = 18591861.0

# 2. Identify the rows containing the highly improbable max date
outlier_rows = df[df['Object End Date'] == suspicious_max_date]

# 3. Print details of the identified outlier(s)
if not outlier_rows.empty:
    print(f"Found {len(outlier_rows)} records with the extreme Outlier Date: {suspicious_max_date}")
    print("--- Outlier Details (likely data entry error) ---")

    # Show the Object ID, Department, and the actual Object Date string (for manual validation)
    print(outlier_rows[['Object ID', 'Department', 'Title', 'Object Date', 'Object End Date']])

    # --- LOF Conceptual Application ---
    # In a real LOF application, we would use Object Begin/End Date as features
    # and fit the LOF model to get anomaly scores.
    # For instance:
    # X = df[['Object Begin Date', 'Object End Date']].values
    # clf = LocalOutlierFactor(n_neighbors=20, contamination='auto')
    # y_pred = clf.fit_predict(X)
    # df['lof_score'] = clf.negative_outlier_factor_
    # print("\nTop 5 LOF Outliers (lowest scores):")
    # print(df.sort_values('lof_score').head(5)[['Object ID', 'Object Begin Date', 'Object End Date', 'lof_score']])

else:
    print("No records found matching the suspicious maximum date.")


STEP 6: Outlier Identification and Inspection (LOF Concept)
Found 1 records with the extreme Outlier Date: 18591861.0
--- Outlier Details (likely data entry error) ---
        Object ID           Department  \
400509     653050  Drawings and Prints   

                                                    Title Object Date  \
400509  Left hand side of sheet 'The world upside dow'...   1859 1861   

        Object End Date  
400509         18591861  


In [6]:
# --- Data Transformation: Fixing Date Outlier ---
print("\n" + "="*50)
print("STEP 7: Data Transformation & Consistency Check")
print("="*50)

# 1. Correct the known extreme outlier (18591861 -> 1861)
outlier_value = 18591861
correct_value = 1861

df.loc[df['Object End Date'] == outlier_value, 'Object End Date'] = correct_value
print(f"Corrected {df[df['Object End Date'] == outlier_value].shape[0]} outlier(s): {outlier_value} -> {correct_value}")

# Re-run descriptive statistics to confirm the max is now fixed
print("\nDescriptive Stats After Outlier Correction:")
print(df[['Object Begin Date', 'Object End Date']].describe())


# 2. Check for Date Inconsistency (End Date < Begin Date)
inconsistent_rows = df[df['Object End Date'] < df['Object Begin Date']]
inconsistent_count = inconsistent_rows.shape[0]

if inconsistent_count > 0:
    print(f"\nFound {inconsistent_count} records where End Date < Begin Date (DQ Issue):")
    # Show the IDs and dates of the first 5 inconsistent records
    print(inconsistent_rows[['Object ID', 'Object Begin Date', 'Object End Date', 'Title']].head())
else:
    print("\nDate Consistency Check Passed: Object End Date is never before Object Begin Date.")


STEP 7: Data Transformation & Consistency Check
Corrected 0 outlier(s): 18591861 -> 1861

Descriptive Stats After Outlier Correction:
       Object Begin Date  Object End Date
count       4.482030e+05    448203.000000
mean        1.390817e+03      1451.969614
std         2.779816e+04      1018.678307
min        -9.500000e+04    -35000.000000
25%         1.567000e+03      1600.000000
50%         1.800000e+03      1850.000000
75%         1.895000e+03      1910.000000
max         1.859186e+07    157555.000000

Found 196 records where End Date < Begin Date (DQ Issue):
        Object ID  Object Begin Date  Object End Date  \
32769       38468                -45              -55   
47451       55128               1861                9   
81309      107853               1875             1800   
121753     209514               1785             1779   
139421     229879               1785             1773   

                                     Title  
32769                                  N

In [7]:
# --- Data Transformation: Fixing New Outlier and Inconsistencies ---
print("\n" + "="*50)
print("STEP 8: Data Transformation & Consistency Correction")
print("="*50)

# 1. Identify and Inspect the next extreme Outlier (Object End Date = 157555)
extreme_outlier_rows = df[df['Object End Date'] == df['Object End Date'].max()]

if not extreme_outlier_rows.empty:
    print(f"Inspecting New Extreme Outlier (Max End Date: {df['Object End Date'].max()}):")
    print(extreme_outlier_rows[['Object ID', 'Title', 'Object Date', 'Object End Date']])

    # We will assume this object should not exist in the future, so we cap it.
    # We will cap all dates > 2025 to 2025 to manage the large noise.
    current_year = 2025
    df.loc[df['Object End Date'] > current_year, 'Object End Date'] = current_year

    # Recount affected records
    capped_count = (extreme_outlier_rows['Object End Date'] > current_year).sum()
    print(f"\nCapped {capped_count} records where Object End Date was > {current_year} to {current_year}.")


# 2. Fix Inconsistent Dates (End Date < Begin Date)
inconsistent_rows = df['Object End Date'] < df['Object Begin Date']
inconsistent_count = inconsistent_rows.sum()

if inconsistent_count > 0:
    # Use tuple assignment to swap the values efficiently for inconsistent rows
    # This applies to the entire DataFrame in a single operation

    # Create temporary variables for swapping
    begin_temp = df.loc[inconsistent_rows, 'Object Begin Date'].copy()
    end_temp = df.loc[inconsistent_rows, 'Object End Date'].copy()

    # Perform the swap: End Date gets the old Begin Date, Begin Date gets the old End Date
    df.loc[inconsistent_rows, 'Object End Date'] = begin_temp
    df.loc[inconsistent_rows, 'Object Begin Date'] = end_temp

    print(f"\nCorrected {inconsistent_count} records by swapping Begin and End Dates.")

    # Verify correction
    print("\nVerification (Inconsistent Check After Swap):")
    print(df[df['Object End Date'] < df['Object Begin Date']].shape[0])
else:
    print("\nDate Consistency Check Passed after initial transformations.")


STEP 8: Data Transformation & Consistency Correction
Inspecting New Extreme Outlier (Max End Date: 157555):
        Object ID                        Title  Object Date  Object End Date
112028     198783  Storage jar (one of a pair)  ca. 1550–70           157555

Capped 1 records where Object End Date was > 2025 to 2025.

Corrected 196 records by swapping Begin and End Dates.

Verification (Inconsistent Check After Swap):
0


In [8]:
# --- Preparation for Deduplication (Lecture 9) ---
print("\n" + "="*50)
print("STEP 9: Checking for Duplicate Records")
print("="*50)

# 1. Check for Exact (Hard) Duplicates
hard_duplicates_count = df.duplicated().sum()
print(f"Total number of exact (hard) duplicate rows: {hard_duplicates_count}")

# 2. Check for Duplicates based on Primary Identifiers
# Object ID should be unique, so any duplicates here are critical DQ errors.
id_duplicates = df[df.duplicated(subset=['Object ID'], keep=False)].sort_values('Object ID')
id_duplicates_count = id_duplicates.shape[0]

if id_duplicates_count > 0:
    print(f"Found {id_duplicates_count} rows with duplicate Object IDs (CRITICAL DQ ISSUE):")
    print(id_duplicates[['Object ID', 'Object Number', 'Title', 'Department']].head(10))
else:
    print("No critical duplicates found based on Object ID.")


STEP 9: Checking for Duplicate Records
Total number of exact (hard) duplicate rows: 0
No critical duplicates found based on Object ID.


In [9]:
# --- Deduplication (Soft Duplicates Check) ---
print("\n" + "="*50)
print("STEP 10: Checking for Soft Duplicates (Title Field)")
print("="*50)

# 1. Prepare the Title column for fuzzy matching:
#    Convert to lowercase and remove common trailing spaces/punctuation.
df['Title_Clean'] = df['Title'].str.lower().str.strip().str.replace(r'[.,;:]$', '', regex=True)

# 2. Count duplicate titles (where one title may belong to multiple objects,
#    e.g., a set of prints, or actual duplicates)
title_counts = df['Title_Clean'].value_counts()

# Exclude titles that only appear once (they cannot be duplicates)
potential_duplicates = title_counts[title_counts > 1]
print(f"Total number of unique titles that appear more than once (potential soft duplicates/sets): {len(potential_duplicates)}")
print(f"Total objects involved in potential soft duplicates/sets: {potential_duplicates.sum()}")

# 3. Inspect the top 5 most common titles
print("\nTop 5 Most Frequent Titles (Potential for Sets/Duplicates):")
top_titles = potential_duplicates.head(5).index.tolist()

for title in top_titles:
    print(f"\n--- Title: '{title}' ---")
    # Show the relevant columns for inspection
    print(df[df['Title_Clean'] == title][['Object ID', 'Object Number', 'Title', 'Object Name', 'Department']].head(3))

# 4. Cleanup temporary column
df = df.drop(columns=['Title_Clean'])


STEP 10: Checking for Soft Duplicates (Title Field)
Total number of unique titles that appear more than once (potential soft duplicates/sets): 28225
Total objects involved in potential soft duplicates/sets: 221482

Top 5 Most Frequent Titles (Potential for Sets/Duplicates):

--- Title: 'fragment' ---
      Object ID Object Number     Title Object Name                Department
3743       3995      37.132.1  Fragment    Fragment  American Decorative Arts
3744       3996      37.132.2  Fragment    Fragment  American Decorative Arts
3745       3997      37.132.3  Fragment    Fragment  American Decorative Arts

--- Title: 'piece' ---
       Object ID Object Number  Title Object Name                Department
12551      13673       1989.65  Piece       Piece  American Decorative Arts
12559      13687        32.113  Piece       Piece  American Decorative Arts
12587      13715       36.89.1  Piece       Piece  American Decorative Arts

--- Title: 'dress' ---
       Object ID Object Number  T

In [10]:
# --- Final Data Transformation & Process Improvement (Standardization) ---
print("\n" + "="*50)
print("STEP 11: Data Fusion Preparation & Standardization")
print("="*50)

# RE-CREATING THE CLEAN COLUMN (CORRECTION FOR KEYERROR)
df['Title_Clean'] = df['Title'].str.lower().str.strip().str.replace(r'[.,;:]$', '', regex=True)

# Check for the most generic titles which could be improved or standardized
generic_titles = ['fragment', 'piece', 'drawing']

for title in generic_titles:
    # 1. Check how many of these generic titles have a blank/null 'Object Date'
    #    (This is a Data Completeness check on poorly described items)

    # We must use .fillna('') for the title column when creating the subset
    # to avoid errors if the 'Title_Clean' is NaN for some reason,
    # but the primary issue is just filtering.
    subset = df[df['Title_Clean'] == title]
    missing_date_count = subset['Object Date'].isnull().sum()

    print(f"\nTitle: '{title}' (Total: {len(subset)} records)")
    print(f"- Missing 'Object Date' (Completeness Check): {missing_date_count}")

    # 2. Simple Transformation: If Title is too generic, copy a less generic field.
    #    (Conceptual improvement: If Title is 'Fragment', perhaps the Object Name is more specific)

    # Note: We must handle potential NaN values in 'Object Name' for string comparison
    comparison_subset = subset[subset['Object Name'].str.lower().fillna('') != title]
    print(f"- Records where Object Name is DIFFERENT from '{title}': {len(comparison_subset)}")

    # In a real fusion process, these records would be flagged for manual review or batch update
    if len(comparison_subset) > 0:
         # Check if comparison_subset is not empty before trying to access iloc[0]
         print(f"  Example: Object Name '{comparison_subset['Object Name'].iloc[0]}' vs Title '{title}'")

# Cleanup: Drop the temporary clean column used for analysis
df = df.drop(columns=['Title_Clean'])


STEP 11: Data Fusion Preparation & Standardization

Title: 'fragment' (Total: 7087 records)
- Missing 'Object Date' (Completeness Check): 9
- Records where Object Name is DIFFERENT from 'fragment': 15
  Example: Object Name 'Textile fragment' vs Title 'fragment'

Title: 'piece' (Total: 4690 records)
- Missing 'Object Date' (Completeness Check): 4
- Records where Object Name is DIFFERENT from 'piece': 87
  Example: Object Name 'Printed piece' vs Title 'piece'

Title: 'drawing' (Total: 30 records)
- Missing 'Object Date' (Completeness Check): 0
- Records where Object Name is DIFFERENT from 'drawing': 19
  Example: Object Name 'Drawing, Fraktur' vs Title 'drawing'


In [11]:
# --- Data Transformation: Standardization of Generic Titles ---
print("\n" + "="*50)
print("STEP 12: Executing Standardization Transformation")
print("="*50)

# 1. Re-create the cleaned column temporarily for filtering
# Note: We must re-create this column to avoid the previous KeyError.
df['Title_Clean'] = df['Title'].str.lower().str.strip().str.replace(r'[.,;:]$', '', regex=True)

# Define the generic titles identified
generic_titles = ['fragment', 'piece', 'drawing']

# 2. Identify all rows needing correction
# Condition A: Title_Clean is one of the generic_titles
# Condition B: Object Name exists (not null) AND Object Name (lowercase) is NOT equal to the generic title
# We use .isin() for efficiency.
correction_mask = (df['Title_Clean'].isin(generic_titles)) & \
                  (df['Object Name'].notnull()) & \
                  (df['Object Name'].str.lower() != df['Title_Clean'])

# Count the records that will be corrected
records_to_correct = correction_mask.sum()

if records_to_correct > 0:
    # 3. Execute the transformation: Copy the value from 'Object Name' to 'Title'
    df.loc[correction_mask, 'Title'] = df.loc[correction_mask, 'Object Name']

    print(f"✅ Successfully corrected {records_to_correct} records.")
    print("   The 'Title' field is now updated with the more specific 'Object Name'.")

    # 4. Verification Check: Show a corrected record example
    example_correction = df[correction_mask].iloc[0]
    print("\n--- Example of Corrected Record ---")
    print(f"Object ID: {example_correction['Object ID']}")
    print(f"New Title: '{example_correction['Title']}'")
    print(f"Old Clean Title (Verification): '{example_correction['Title_Clean']}'")

else:
    print("No additional generic titles needed standardization after initial checks.")

# 5. Cleanup
df = df.drop(columns=['Title_Clean'])


STEP 12: Executing Standardization Transformation
✅ Successfully corrected 121 records.
   The 'Title' field is now updated with the more specific 'Object Name'.

--- Example of Corrected Record ---
Object ID: 3408
New Title: 'Drawing, Fraktur'
Old Clean Title (Verification): 'drawing'


In [12]:
df.head()

,Object Number,Is Highlight,Is Public Domain,Object ID,Department,Object Name,Title,Culture,Period,Dynasty,...,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Metadata Date,Repository
0,1979.486.1,False,False,1,American Decorative Arts,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/1,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
1,1980.264.5,False,False,2,American Decorative Arts,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/2,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
2,67.265.9,False,False,3,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/3,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
3,67.265.10,False,False,4,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/4,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
4,67.265.11,False,False,5,American Decorative Arts,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Metal,NaN,http://www.metmuseum.org/art/collection/search/5,4/3/2017 8:00:08 AM,"Metropolitan Museum of Art, New York, NY"
